<a href="https://colab.research.google.com/github/Fawzy-AI-Explorer/X-From-Scratch/blob/main/Linear_Regression-From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression From Scratch

## Goals
In this toturial, we will:

- Implement the linear regression model
- Implement the gradient descent algorithm to train the model
- Implement $R^2 Score$ to calculate the accuracy<br><br>

*All from scratch*

## Tools
In this toturial, we will make use of:
- math, This module provides access to the mathematical functions defined by the C standard
- pandas, a Python library used for working with data sets
- NumPy, a popular library for scientific computing
- seaborn, a Python data visualization library based on matplotlib
- Matplotlib, a popular library for plotting data

In [126]:
import math, copy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [127]:
"""Define a linear regressino class."""


class LinearRegression():
    """Representation of a linear regression model.
    the model predicts a number form  infinitely many possible outputs.
    """

    def __init__(self, learning_rate=1e-4, max_iter=1000):
      """Initialize the linear regression model

        Args:
          learning_rage (float) : The number indicates the step in gradient descent.
          max_iter (int) : The maximum number of passes over the training data.
      """

      self.learning_rate = learning_rate
      self.max_iter = max_iter
      self.w, self.b = None, 0.

    def fit(self, X, y):
      """
      Performs batch gradient descent to learn w and b. Updates w and b by taking
      num_iters gradient steps with learning rate alpha

      Args:
        X (ndarray (m,n))   : Data, m examples with n features
        y (ndarray (m,))    : target values

      Returns:
        self (object) : Fitted model estimator.
        """

      # An array to store cost J and w's at each iteration primarily for graphing later
      J_history = []
      W_history = []
      b_history = []
      n = 1
      if (len(X.shape) > 1):
        n = X.shape[1]
      self.w = np.zeros((n, ))
      self.b = 0.

      for i in range(self.max_iter):
        dj_dw, dj_db = self.compute_gradient(X, y)
        self.w = self.w - (self.learning_rate * dj_dw)
        self.b -= self.learning_rate * dj_db

        if i < 100000:
          J_history.append(self.compute_cost(X, y))
          W_history.append(copy.deepcopy(self.w))
          b_history.append(copy.deepcopy(self.b))

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i % math.ceil(self.max_iter / 10) == 0:
          print("{:>8}  cost={:>1.5e}".format(i, self.compute_cost(X, y)), end='')
          for i in range(n):
            print("  w[{}]={:>1.1e}".format(i, self.w[i]), end='')
          print("  b={:>1.1e}".format(self.b), end='')
          for i in range(n):
            print("  dj_dw[{}]={:>1.1e}".format(i, dj_dw[i]), end='')
          print("  dj_db={:>1.1e}".format(dj_db))

      return self


    def compute_gradient(self, X, y):
      """
      Computes the gradient for linear regression
      Args:
        X (ndarray (m,n)): Data, m examples with n features
        y (ndarray (m,)) : target values

      Returns:
        dj_dw (ndarray (n,)): The gradient of the cost w.r.t. the parameters w.
        dj_db (scalar):       The gradient of the cost w.r.t. the parameter b.
      """

      m = X.shape[0]
      n = 1
      if (len(X.shape) > 1):
        n = X.shape[1]
      dj_dw = np.zeros((n,))
      dj_db = 0.

      for i in range(m):
        f_wb_i = self.predict(X[i])
        err = (f_wb_i - y[i])
        dj_dw = dj_dw + (err * X[i])
        dj_db += err

      dj_dw /= m
      dj_db /= m

      return dj_dw, dj_db


    def compute_cost(self, X, y):
      """
      Computes the cost function for linear regression.

      Args:
        X (ndarray (m,n)): Data, m examples and n features
        y (ndarray (m,)): target values

      Returns
          total_cost (float): The cost of using w,b as the parameters for linear regression
                to fit the data points in x and y
      """

      m = X.shape[0]
      cost = 0.0
      for i in range(m):
        f_wb_i = self.predict(X[i])
        cost += (f_wb_i - y[i]) ** 2

      cost /= (2 * m)

      return cost

    def predict(self, X):
      """
      single predict using linear regression
      Args:
        X (ndarray): Shape (m, n) example with multiple features

      Returns:
        p (scalar):  prediction
      """

      p = np.dot(X, self.w) + self.b
      return p

    def score(self, X, y):
      """
      Returns the accuracy of the model
      Args:
      X (ndarray): Shape(m, n) examples with multiple features
      y (ndarray): Shape (m,) the actual target values

      Returns:
        accuracy (scalar): the accuracy of the model
      """
      m = X.shape[0]
      n = 1
      if (len(X.shape) > 1):
        n = X.shape[1]
      RSS = 0.
      TSS = 0.
      y_mu = np.mean(y)

      for i in range(m):
          RSS += (y[i] - self.predict(X[i])) ** 2
          TSS += (y[i] - y_mu) ** 2

      score = 1 - RSS / TSS
      return round(score, 2)

    def get_params(self):
      """
      Get parameters for this estimator.

      Returns:
        params (dict) : Parameter names mapped to their values.
      """
      params = {}
      params["learning_rate"] = self.learning_rate
      params["max_iter"] = self.max_iter
      if self.w is not None:
        for i in range(len(self.w)):
            p_name = "w" + str(i)
            params[p_name] = self.w[i]
      if self.b is not None:
        params["b"] = self.b

      return params

    def set_params(self, **params):
      """
      Set the parameters of this estimator.

      Args:
        Estimator parameters.

      Returns:
        self (estimator instance) : Estimator instance.
      """


Now let's test the model

In [128]:
# Load the data set
url = 'https://raw.githubusercontent.com/Ad7amstein/Linear_Regression-E-commerce/main/Ecommerce%20Customers.csv'
data = pd.read_csv(url)
# extract the four main features
X = data.values[:, 3:7]
y = data.values[:, 7]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
lr2 = LinearRegression(max_iter=10000, learning_rate=1e-4)
lr2.fit(X_train, y_train)
print("The score of our model is {}%".format(lr2.score(X_test, y_test) * 100))

       0  cost=7.18746e+04  w[0]=1.7e+00  w[1]=6.1e-01  w[2]=1.9e+00  w[3]=1.9e-01  b=5.0e-02  dj_dw[0]=-1.7e+04  dj_dw[1]=-6.1e+03  dj_dw[2]=-1.9e+04  dj_dw[3]=-1.9e+03  dj_db=-5.0e+02
    1000  cost=2.49010e+03  w[0]=6.9e+00  w[1]=5.5e+00  w[2]=4.9e+00  w[3]=6.9e+00  b=1.5e-01  dj_dw[0]=-5.7e+00  dj_dw[1]=-3.0e+01  dj_dw[2]=2.1e+01  dj_dw[3]=-5.9e+01  dj_db=4.4e-01
    2000  cost=2.05676e+03  w[0]=7.5e+00  w[1]=8.4e+00  w[2]=3.0e+00  w[3]=1.2e+01  b=1.0e-01  dj_dw[0]=-5.1e+00  dj_dw[1]=-2.7e+01  dj_dw[2]=1.9e+01  dj_dw[3]=-5.3e+01  dj_db=4.4e-01
    3000  cost=1.70796e+03  w[0]=8.0e+00  w[1]=1.1e+01  w[2]=1.2e+00  w[3]=1.7e+01  b=5.9e-02  dj_dw[0]=-4.5e+00  dj_dw[1]=-2.5e+01  dj_dw[2]=1.7e+01  dj_dw[3]=-4.7e+01  dj_db=4.4e-01
    4000  cost=1.42722e+03  w[0]=8.4e+00  w[1]=1.3e+01  w[2]=-3.7e-01  w[3]=2.2e+01  b=1.5e-02  dj_dw[0]=-4.0e+00  dj_dw[1]=-2.2e+01  dj_dw[2]=1.5e+01  dj_dw[3]=-4.2e+01  dj_db=4.4e-01
    5000  cost=1.20125e+03  w[0]=8.8e+00  w[1]=1.5e+01  w[2]=-1.8e+00  w[3]=2